<a href="https://colab.research.google.com/github/shalvamist/GPTme/blob/main/Examples/notebooks/Running_LlamaCPP_in_Colab_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" pip install llama-cpp-python
!pip install huggingface_hub

In [1]:
%%shell
git clone https://github.com/ggerganov/llama.cpp
export LLAMA_CUBLAS=on

cd llama.cpp
make clean
make libllama.so

export LLAMA_CPP_LIB=./llama.cpp/libllama.so

Cloning into 'llama.cpp'...
remote: Enumerating objects: 20318, done.
remote: Counting objects: 100% (7066/7066), done.
remote: Compressing objects: 100% (445/445), done.
remote: Total 20318 (delta 6859), reused 6667 (delta 6618), pack-reused 13252
Receiving objects: 100% (20318/20318), 23.69 MiB | 22.22 MiB/s, done.
Resolving deltas: 100% (14359/14359), done.
I ccache not found. Consider installing it for faster compilation.
I llama.cpp build info: 
I UNAME_S:   Linux
I UNAME_P:   x86_64
I UNAME_M:   x86_64
I CFLAGS:    -I. -Icommon -D_XOPEN_SOURCE=600 -D_GNU_SOURCE -DNDEBUG -DGGML_USE_CUBLAS -I/usr/local/cuda/include -I/usr/local/cuda/targets/x86_64-linux/include  -std=c11   -fPIC -O3 -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wshadow -Wstrict-prototypes -Wpointer-arith -Wmissing-prototypes -Werror=implicit-int -Werror=implicit-function-declaration -pthread -march=native -mtune=native -Wdouble-promotion 
I CXXFLAGS:  -std=c++11 -fPIC -O3 -Wall -Wextra -Wpedantic -Wcas

In [2]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on -DCUDA_PATH=/usr/local/cuda-12.2 -DCUDAToolkit_ROOT=/usr/local/cuda-12.2 -DCUDAToolkit_INCLUDE_DIR=/usr/local/cuda-12/include -DCUDAToolkit_LIBRARY_DIR=/usr/local/cuda-12.2/lib64 -DCMAKE_CUDA_COMPILER:PATH=/usr/local/cuda/bin/nvcc" FORCE_CMAKE=1 pip install --upgrade --force-reinstall llama-cpp-python --no-cache-dir --verbos

Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.9/36.9 MB 86.3 MB/s eta 0:00:00
  Running command pip subprocess to install build dependencies
  Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
    Using cached scikit_build_core-0.8.2-py3-none-any.whl (140 kB)
    Using cached exceptiongroup-1.2.0-py3-none-any.whl (16 kB)
    Using cached packaging-24.0-py3-none-any.whl (53 kB)
    Using cached tomli-2.0.1-py3-none-any.whl (12 kB)
    Using cached pathspec-0.12.1-py3-none-any.whl (31 kB)
    Using cached pyproject_metadata-0.7.1-py3-none-any.whl (7.4 kB)
  Installing build dependencies ... done
  Running command Getting requirements to build wheel
  Getting requirements to build wheel ... done
  Running command pip subprocess to install backend dependencies
  Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
    Using cached cmake-3.28.3-py2.py3-

In [3]:
from huggingface_hub import hf_hub_download

REPO_ID = "TheBloke/CapybaraHermes-2.5-Mistral-7B-GGUF"
FILENAME = "capybarahermes-2.5-mistral-7b.Q6_K.gguf"

model_path = hf_hub_download(local_dir='/content',repo_id=REPO_ID, filename=FILENAME)

from llama_cpp import Llama
llm = Llama(
      model_path=model_path,
      n_ctx=32768,  # The max sequence length to use - note that longer sequence lengths require much more resources
      n_threads=8,            # The number of CPU threads to use, tailor to your system and the resulting performance
      n_gpu_layers=35         # The number of layers to offload to GPU, if you have GPU acceleration available
)

output = llm(
      "Q: Name the most important roman emperor? A: ", # Prompt
      max_tokens=32, # Generate up to 32 tokens, set to None to generate up to the end of the context window
      stop=["Q:", "\n"], # Stop generating just before the model would generate a new question
      echo=True # Echo the prompt back in the output
) # Generate a completion, can also call create_completion
output

llama_model_loader: loaded meta data with 24 key-value pairs and 291 tensors from /content/capybarahermes-2.5-mistral-7b.Q6_K.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = argilla_capybarahermes-2.5-mistral-7b
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:   

{'id': 'cmpl-633d4d06-bbed-43ca-a132-79104d493439',
 'object': 'text_completion',
 'created': 1710390126,
 'model': '/content/capybarahermes-2.5-mistral-7b.Q6_K.gguf',
 'choices': [{'text': 'Q: Name the most important roman emperor? A:  The most important Roman Emperor is considered to be Augustus (27 BCE – 14 CE), who founded the Roman Empire and established the Pax',
   'index': 0,
   'logprobs': None,
   'finish_reason': 'length'}],
 'usage': {'prompt_tokens': 13, 'completion_tokens': 32, 'total_tokens': 45}}